In [3]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm

from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

## 1. Process the Mueller Matrices of the folders

In [4]:
# set the parameters to run the script
data_folder_path = r'C:\Users\romai\Desktop\Freezing_dev'
directories = [data_folder_path]
calib_directory = r'C:\Users\romai\Desktop\calib'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [5]:
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:44<00:00,  2.45s/it]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [6]:
import time

# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    propagation_lists = {}
    
    for tissue_type in tissue_types:
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_lists[tissue_type] = selection_of_ROIs.square_selection(path_folder_temp, path_folder, WM, 
                                                                            tissue_type, path_align_folder)
    # 3. actually do the alignment
    propagation_lists = automatic_ROI_propagation.generate_combined_mask(propagation_lists, path_align_folder)
    automatic_ROI_propagation.do_alignment(path_align_folder)
    output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
    
    # 4. and collect the data after propagation
    for tissue_type in tissue_types:
        WM = tissue_type == 'WM'
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_lists[tissue_type], output_folders,
                                                       data_folder_path)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Folder already exists, trying again...


 17%|█████████████▊                                                                     | 1/6 [03:04<15:22, 184.45s/it]Traceback (most recent call last):
  File "C:\Users\romai\Documents\FreezingEffect\src\freezingeffect\selection_of_ROIs.py", line 395, in square_selection
    coordinates_long, grid = get_square_coordinates(mask_matter, param_ROIs['square_size'], grid, MM = MM)
TypeError: cannot unpack non-iterable NoneType object
 67%|███████████████████████████████████████████████████████▎                           | 4/6 [12:12<06:08, 184.10s/it]

Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...


 83%|█████████████████████████████████████████████████████████████████████▏             | 5/6 [14:55<02:56, 176.60s/it]

Folder already exists, trying again...
Folder already exists, trying again...


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [17:35<00:00, 175.85s/it]


## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [11]:
from freezingeffect import parameter_comparaison

In [12]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_, max_nb_of_squares = parameter_comparaison.get_parameters()
max_nb_of_squares = max_nb_of_squares * len(base_dirs)


# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

['C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_1',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_10',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_11',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_12',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_13',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_5',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_6',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_7',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR

#### 4.2. Get the data for the measurement

In [13]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['GM'][list(data_fixation['GM'].keys())[0]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,103.245648,24.097137,105.168539,103.245648,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.850209,0.016843,0.849367,0.850455,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.034988,0.010268,0.030612,0.035489,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,11.955835,4.508347,11.392405,12.005583,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,3.486289,12.215557,4.044944,3.486289,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.789501,0.030668,0.765823,0.786918,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.029731,0.010291,0.020408,0.029379,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,13.281737,4.611449,10.253165,12.207735,None
8,2023-04-12_T_FRE-FR-T2_FX_S1_1,azimuth,154.375915,23.856503,151.685393,154.375915,None
9,2023-04-12_T_FRE-FR-T2_FX_S1_1,depolarization,0.834809,0.018121,0.821519,0.834369,None


In [14]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [15]:
data_fixation['GM'][list(data_fixation['GM'].keys())[0]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,103.245648,24.097137,105.168539,103.245648,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.850209,0.016843,0.849367,0.850455,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.034988,0.010268,0.030612,0.035489,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,11.955835,4.508347,11.392405,12.005583,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,3.486289,12.215557,4.044944,3.486289,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.789501,0.030668,0.765823,0.786918,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.029731,0.010291,0.020408,0.029379,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,13.281737,4.611449,10.253165,12.207735,None
8,2023-04-12_T_FRE-FR-T2_FX_S1_1,azimuth,154.375915,23.856503,151.685393,154.375915,None
9,2023-04-12_T_FRE-FR-T2_FX_S1_1,depolarization,0.834809,0.018121,0.821519,0.834369,None


In [16]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1', 'T2']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [17]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
0,13.280640,19.187782,16.542116,14.916239,19.312810,26.490899,12.952672,11.784682,17.909897,11.914906,...,15.167666,14.707312,30.482439,14.914871,12.694514,25.384967,16.296715,18.467797,14.486394,23.430000
1,13.189990,26.819185,12.726576,13.095968,11.991299,14.553693,15.585741,NaN,13.214168,12.791274,...,11.447135,16.004091,14.556841,15.838989,16.086709,13.249207,10.194272,18.246567,NaN,21.414212
2,19.890217,24.269055,20.197616,12.839839,12.610759,20.044548,27.488406,NaN,13.794904,18.701670,...,13.705923,14.432045,12.364349,10.422545,10.977096,10.256290,12.381990,14.236722,NaN,14.696490


In [18]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000
1,0.993174,1.397722,0.769344,0.877967,0.620899,0.549385,1.203284,NaN,0.737814,1.073552,...,0.754706,1.088172,0.477548,1.061959,1.267217,0.521931,0.625542,0.988021,NaN,0.913965
2,1.497685,1.264818,1.220981,0.860796,0.652974,0.756658,2.122219,NaN,0.770239,1.569603,...,0.903628,0.981284,0.405622,0.698802,0.864712,0.404030,0.759784,0.770894,NaN,0.627251


#### 4.3. Statistical analysis

In [19]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [20]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z       p_value    n_1    n_2  before/GM  \
parameter      data_type time                                                   
azimuth        GM        T0        0.0  2.533117e-51  133.0  133.0   0.000000   
                         T1     8844.5  1.000000e+00  133.0  133.0   0.000000   
                         T2        0.0  1.470012e-50  133.0  126.0   0.000000   
               WM        T0        0.0  5.992085e-51  132.0  132.0   0.000000   
                         T1     8712.0  1.000000e+00  132.0  132.0   0.000000   
                         T2        0.0  1.263420e-50  132.0  129.0   0.000000   
depolarization GM        T0    11400.0  2.463024e-02  133.0  150.0   1.008988   
                         T1     8844.5  1.000000e+00  133.0  133.0   1.008988   
                         T2     2295.0  4.475349e-25  133.0  131.0   1.008988   
               WM        T0    12410.0  7.445075e-06  132.0  146.0   1.003769   
                         T1     8712.0  1.000000e+00  132.0  132.0   1.003769   
                         T2    11313.0  2.761609e-05  132.0  132.0   1.003769   
retardance     GM        T0     6300.0  6.769899e-09  133.0  150.0   0.827716   
                         T1     8844.5  1.000000e+00  133.0  133.0   0.827716   
                         T2    10482.0  4.323909e-03  133.0  131.0   0.827716   
               WM        T0     9344.0  6.377020e-01  132.0  146.0   0.993699   
                         T1     8712.0  1.000000e+00  132.0  132.0   0.993699   
                         T2     9730.0  1.009390e-01  132.0  132.0   0.993699   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0         0.000000  32.778059     24.758623  
                         T1         0.000000   0.000000      0.000000  
                         T2         0.000000  18.743690     18.171671  
               WM        T0         0.000000  27.561074     23.706571  
                         T1         0.000000   0.000000      0.000000  
                         T2         0.000000  19.153476     18.589341  
depolarization GM        T0         0.043066   1.000000      0.000000  
                         T1         0.043066   1.008988      0.043066  
                         T2         0.043066   1.082692      0.050968  
               WM        T0         0.011716   1.000000      0.000000  
                         T1         0.011716   1.003769      0.011716  
                         T2         0.011716   0.998298      0.013085  
retardance     GM        T0         0.422567   1.000000      0.000000  
                         T1         0.422567   0.827716      0.422567  
                         T2         0.422567   0.691029      0.359992  
               WM        T0         0.278909   1.000000      0.000000  
                         T1         0.278909   0.993699      0.278909  
                         T2         0.278909   0.966208      0.375564

In [21]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [22]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_nb_of_squares)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_nb_of_squares)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_nb_of_squares)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_nb_of_squares)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_nb_of_squares)

retardance_cv

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,NaN,NaN,NaN,NaN
1,1.016838,0.756413,1.391658,0.643034,1.142495,NaN,NaN,NaN,0.701738,0.557905,...,1.267217,0.521931,0.625542,0.988021,NaN,0.913965,NaN,NaN,NaN,NaN
2,0.657287,0.337255,1.172699,0.422487,NaN,NaN,NaN,NaN,0.404995,0.379320,...,0.864712,0.404030,0.759784,0.770894,NaN,0.627251,NaN,NaN,NaN,NaN


In [23]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))